In [36]:
import json
from langchain_groq import ChatGroq
from langchain.schema import SystemMessage, HumanMessage
from langchain.prompts import ChatPromptTemplate
import os 
import re

In [37]:
from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv("Groq_API_KEY")

In [38]:
chat_model = ChatGroq(model="mixtral-8x7b-32768", api_key=api_key)

In [39]:
def extract_json(response_text):
    """ Extract JSON from AI response using regex """
    match = re.search(r"```json\n(.*?)\n```", response_text, re.DOTALL)
    if match:
        try:
            return json.loads(match.group(1))  # Convert extracted JSON string to dictionary
        except json.JSONDecodeError:
            return {"error": "Invalid JSON format", "raw_response": response_text}
    return {"error": "No JSON found in response", "raw_response": response_text}

In [40]:
# Define the Chain of Thought (CoT) prompt
cot_prompt = ChatPromptTemplate.from_messages([
    ("system", """
    You are an experienced data scientist with expertise in solving complex business problems.
    Your goal is to guide the client through different approaches to solving their problem.

    ### **Instructions:**
    Follow these structured steps to analyze and provide a solution:

    1. **Understand the Problem** - Analyze the client's question and break it down.
    2. **Existing Solutions** - Identify predefined solutions available in the market.
    3. **Alternative Approaches** - Explore different ways to solve the problem.
    4. **Final Recommendation** - Provide the best solution.

    ---

    ### **Example Walkthrough**
    
    #### **Step 1: Understanding the Problem**
    **THOUGHT:** The client needs to analyze customer churn using historical data.

    #### **Step 2: Checking for Existing Solutions**
    **THOUGHT:** Common solutions for churn prediction include logistic regression, decision trees, and neural networks.

    #### **Step 3: Exploring Different Approaches**
    **THOUGHT:** 
    - Approach 1: Use a rule-based model with predefined thresholds.
    - Approach 2: Train a machine learning model using historical churn data.
    - Approach 3: Implement deep learning techniques for better accuracy.

    #### **Step 4: Selecting the Best Approach**
    **ACTION:** Use `train_ml_model_tool` with `algorithm='Random Forest'` and `data='customer_churn.csv'`

    **OBSERVATION:** Model trained with 85% accuracy.

    ### **Final Output Format (JSON)**
    **ONLY return valid JSON** following this structure. Do not include any other text.

    ```json
    {{
        "solution": "Trained a Random Forest model on customer churn data with 85% accuracy.",
        "recommendation": "Recommend further hyperparameter tuning for improvement."
    }}
    ```

    **Do NOT include any text before or after the JSON output.**"""),
    ("user", "{input}")
])

In [44]:
def chatbot():
    print("💬 AI Chatbot is running! Type 'exit' to stop.")

    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            print("👋 Exiting chat. Goodbye!")
            break

        # Format messages with the prompt
        messages = [
            SystemMessage(content=cot_prompt.format(input=user_input)),
            HumanMessage(content=user_input)
        ]

        # Get AI response using LangChain invoke()
        response = chat_model.invoke(messages)
        output_json = extract_json(response.content)

        print(response)
        with open("chatbot_output.json", "w") as json_file:
            json.dump(output_json, json_file, indent=4)

        # Print extracted JSON output
        print(f"🤖 AI: {output_json.get('solution', 'Error in response')}")
        print(f"📌 Recommendation: {output_json.get('recommendation', 'No recommendation available.')}\n")


In [45]:
chatbot()

💬 AI Chatbot is running! Type 'exit' to stop.
content='**Step 1: Understanding the Problem**\n\nThe client wants to create an AI agent, but the specific use case and requirements are not provided.\n\n**Step 2: Checking for Existing Solutions**\n\nThere are several predefined AI agent solutions available in the market, such as:\n\n- Chatbots for customer support and engagement\n- AI-powered virtual assistants for automating tasks\n- AI-driven recommendation systems for personalized experiences\n- AI-based predictive models for decision making\n\n**Step 3: Exploring Different Approaches**\n\n- Approach 1: Develop a rule-based chatbot using a tool like Dialogflow or Microsoft Bot Framework.\n- Approach 2: Implement an AI-powered virtual assistant using pre-built services like Amazon Alexa or Google Assistant.\n- Approach 3: Build a custom AI agent from scratch using machine learning frameworks like TensorFlow or PyTorch.\n\n**Step 4: Selecting the Best Approach**\n\nTo provide a recommend